In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import meteva.method as mem
import meteva.product as mpd
import numpy as np
import datetime
import copy
import matplotlib.pyplot as plt
import pandas as pd

本模块的函数族旨在提供基于带有时空信息的观测预报数据进行分类检验的方法，函数会返回每个分组样本对应的混淆矩阵，频率统计等表格结果，表格会以矩阵形式返回，同时会输出到excel文件当中。在浏览本模块函数说明之前可先参阅函数设计的[关键技术思想](https://www.showdoc.cc/meteva?page_id=4072178802924517)以及本模块测试用的[数据简介](https://www.showdoc.cc/nmc?page_id=3831227192066999)

In [2]:
###################以下开始为数据收集部分的程序

#设置关注的起始时段
time_start = datetime.datetime(2019,1,1,8,0)   
time_end = datetime.datetime(2020,1,1,8,0)
#读取站点列表，并将站点内容为缺省值，当其作为读取站点数据的参数时，如果站点文件中某个站号不存在时,返回结果中该站点保持为缺省值
station = meb.read_stadata_from_micaps3(r"H:\test_data\input\mpd\ob\temp_2m\BT19010102.000")
station.iloc[:,-1] = meb.IV 


##读取收集观测数据
dir_ob = r"H:\test_data\input\mpd\ob\temp_2m\BTYYMMDDHH.000"
sta_list = []
time0 = time_start
while time0 < time_end:
    path = meb.get_path(dir_ob,time0)
    sta = meb.read_stadata_from_micaps3(path,station = station,time = time0,dtime = 0,level = 0,data_name = "ob")
    sta_list.append(sta)
    time0 += datetime.timedelta(hours = 3)
ob_sta_all = pd.concat(sta_list,axis = 0)  #数据拼接
ob_sta_all = meb.sele_by_para(ob_sta_all,value=[-100,100]) #简单的质量控制

#读取收集ec预报数据
dir_ec = r"H:\test_data\input\mpd\ec\temp_2m\YYMMDD\BTYYMMDDHH.TTT"
sta_list =[]
time0 = time_start
while time0 <= time_end:
    for dh in range(0,73,3):
        path = meb.get_path(dir_ec,time0,dh)
        grd = meb.read_griddata_from_micaps4(path)
        if grd is not None:
            sta = meb.interp_gs_linear(grd,station)
            meb.set_stadata_coords(sta,time = time0,dtime = dh,level = 0)
            meb.set_stadata_names(sta,["ecmwf"])
            sta_list.append(sta)
    time0 += datetime.timedelta(hours = 132)  #此处跳着读是为了减少测试用时
ec_sta_all = pd.concat(sta_list,axis = 0)  #数据拼接
            
#读取收集grapes预报数据
dir_grapes = r"H:\test_data\input\mpd\grapes\temp_2m\YYMMDD\BTYYMMDDHH.TTT"
sta_list =[]
time0 = time_start
while time0 <= time_end:
    for dh in range(0,73,3):
        path = meb.get_path(dir_grapes,time0,dh)
        grd = meb.read_griddata_from_micaps4(path)
        if grd is not None:
            sta = meb.interp_gs_linear(grd,station)
            meb.set_stadata_coords(sta,time = time0,dtime = dh,level = 0)
            meb.set_stadata_names(sta,["grapes"])
            sta_list.append(sta)  
    time0 += datetime.timedelta(hours = 132)
grapes_sta_all = pd.concat(sta_list,axis = 0)  #数据拼接
    
#数据匹配合并
sta_all = meb.combine_on_obTime_id(ob_sta_all,[ec_sta_all,grapes_sta_all])
sta_all = meb.sele_by_para(sta_all,drop_IV = True)  #删除包含缺省值的样本
###################以上为数据收集部分的程序

H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.000 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.003 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.006 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.009 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.012 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.015 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.018 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.021 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.024 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.027 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.030 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.033 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.036 is not exist
H:\test_data\input\mpd\ec\temp_2m\190101\BT19010108.039 is not exist
H:\test_data\input\mpd\ec\temp_2m\

H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.000 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.003 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.006 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.009 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.012 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.015 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.018 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.021 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.024 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.027 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.030 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.033 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT19042108.036 is not exist
H:\test_data\input\mpd\grapes\temp_2m\190421\BT1904

# 表格型检验产品生成   
**table(sta_ob_and_fos,method,s = None,g = None,gll = None,save_dir = None,para1 = None)**  
根据输入的站点数据和检验方法，生成表格型检验产品，并自动生成后缀名为.xlsx的文件批量输出到指定目录  


|参数|说明|  
|:--|:--|  
|**sta_ob_and_fos**|实况和预报合并对齐后的数据，形式为站点数据格式如上述例子中的sta_all|   
|**method**|method中的各类表格型的函数名称，例如在本页面中已经目前包括mem.contingency_table_yesorno,   mem.contingency_table_multicategory 和 mem.frequency_table等函数|   
|**s**| 用于选择数据样本的字典参数，具体的参数说明可参见meb.sele_by_dict中的[<font face="黑体" color=red size=5>s</font>](https://www.showdoc.cc/meteva?page_id=3975604785954540)参数  |   
|**g**|用于分组检验的参数，具体用法可参见meb.group中的[<font face="黑体" color=red size=5>g</font>](https://www.showdoc.cc/meteva?page_id=4071849185300418)参数|     
|**gll**| 用于分组检验的参数，具体用法可参见meb.group中的[<font face="黑体" color=red size=5>gll</font>](https://www.showdoc.cc/meteva?page_id=4071849185300418)参数|  
|**save_dir**|生成的表格文件的保存目录|
|**para1**| 检验方法 meteva.method 中ob,fo和save_path三个基本参数之外的第1个参数， 例如在mem.contingency_table_yesorno(ob,fo,grade_list,save_path)中 para1等价于的grade_list |   
|<font face="黑体" color=blue size=3>return</font>|返回一个元组，其包含2个元素，第0个元素为一个矩阵，矩阵内容为每一组样本对应的统计结果合集，元组的第1个元素为实际的分组方式|  
  
**调用示例：**

In [8]:
result,group = mpd.table(sta,mem.contingency_table_yesorno,s = {"month":[12],"id":[54398]})  #选取部分数据进行检验
print(result)  #  shape  = 2 （模式数） * 3 * 3(混淆矩阵shape)
print(group)   # 没有分组

[[[ 86  22 108]
  [  5  27  32]
  [ 91  49 140]]

 [[ 88  26 114]
  [  3  23  26]
  [ 91  49 140]]]
None


In [9]:
result,group = mpd.table(sta,mem.contingency_table_yesorno,g = "hour",
                         save_dir = r"H:\test_data\output\mpd\base\table",para1 = [0,10])  #按小时数（起报时间）分组检验
print(result)  #  shape  = 2(分类数) * 2 （模式数） * 2（等级数）* 3 * 3
print(group)   #  按hour分组，结果包括[8] 和[20]

列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_yesorno_ecmwf_hour8.xlsx
列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_yesorno_grapes_hour8.xlsx
列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_yesorno_ecmwf_hour20.xlsx
列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_yesorno_grapes_hour20.xlsx
[[[[[51.  4. 55.]
    [ 4.  7. 11.]
    [55. 11. 66.]]

   [[66.  0. 66.]
    [ 0.  0.  0.]
    [66.  0. 66.]]]


  [[[52.  2. 54.]
    [ 3.  9. 12.]
    [55. 11. 66.]]

   [[66.  0. 66.]
    [ 0.  0.  0.]
    [66.  0. 66.]]]]



 [[[[33. 20. 53.]
    [ 1. 20. 21.]
    [34. 40. 74.]]

   [[74.  0. 74.]
    [ 0.  0.  0.]
    [74.  0. 74.]]]


  [[[34. 26. 60.]
    [ 0. 14. 14.]
    [34. 40. 74.]]

   [[74.  0. 74.]
    [ 0.  0.  0.]
    [74.  0. 74.]]]]]
[8, 20]


输出的excel文件截图如下：


In [10]:
result,group = mpd.table(sta,mem.contingency_table_multicategory,
                          save_dir = r"H:\test_data\output\mpd\base\table", para1 = [0,5]) 
print(result)  #  shape  = 2 （模式数） * 4（2+等级数）* 4(2+等级数)
print(group)   #  未分组

列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_multicategory_ecmwf.xlsx
列联表已以excel表格形式保存至H:/test_data/output/mpd/base/table/contingency_table_multicategory_grapes.xlsx
[[[ 84.  24.   0. 108.]
  [  5.  14.   8.  27.]
  [  0.   1.   4.   5.]
  [ 89.  39.  12. 140.]]

 [[ 86.  26.   2. 114.]
  [  3.  12.   6.  21.]
  [  0.   1.   4.   5.]
  [ 89.  39.  12. 140.]]]
None


输出的excel截图如下：


In [11]:
result,group = mpd.table(sta,mem.frequency_table, 
                          save_dir = r"H:\test_data\output\mpd\base\table",para1 = [0,1,2,3,4,5]) 
print(result)  #  shape  = 2 （模式数） * 2  * 7(1+等级数)
print(group)   #  未分组

频率统计结果已以excel表格形式保存至H:/test_data/output/mpd/base/table/frequency_table_ecmwf.xlsx
频率统计结果已以excel表格形式保存至H:/test_data/output/mpd/base/table/frequency_table_grapes.xlsx
[[[ 89.   9.  16.   6.   7.   1.  12.]
  [108.  10.   6.   3.   5.   3.   5.]]

 [[ 89.   9.  16.   6.   7.   1.  12.]
  [114.   9.   2.   7.   1.   2.   5.]]]
None


输出excel的截图如下：


从上面的示例中可以看出，基于本程序库提供的函数，实现了数据收集、数据提取和检验计算代码的解耦，使得我们检验分析的过程变得十分灵活，同时解耦也为各部分的进一步模块化提供了基础。值得注意的是采用上述方法进行检验时，会出现如下几个方面的问题：  
    1，将数据全部收集到sta_all变量当中,需要内存足够大  
    2，当sta_all太大时，从中做数据选取，速度会变慢   
    3，不能计算和存储检验中间检验量，也因此不能开展并行计算  
    4，目前只能支持按一个纬度进行简单的分组，不能按多个维度同时分组，例如将样本同时按月份和时效进行分组的功能无法提供。 
上述问题可以在更上一层（perspact）中提供解决方案  